In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, T5ForConditionalGeneration
from transformers import (AdamW, T5Tokenizer, WEIGHTS_NAME,CONFIG_NAME, get_linear_schedule_with_warmup)

ModuleNotFoundError: No module named 'transformers'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')

/Users/kimddogyun/opt/anaconda3/envs/test/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-base')

NameError: name 'T5ForConditionalGeneration' is not defined

In [ ]:
from transformers import T5Config

In [ ]:
config = T5Config.from_pretrained('t5-base')

In [ ]:
print(config)

T5Config {
  "_num_labels": 2,
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "bad_words_ids": null,
  "bos_token_id": null,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "do_sample": false,
  "dropout_rate": 0.1,
  "early_stopping": false,
  "eos_token_id": 1,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_factor": 1.0,
  "is_decoder": false,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_epsilon": 1e-06,
  "length_penalty": 1.0,
  "max_length": 20,
  "min_length": 0,
  "model_type": "t5",
  "n_positions": 512,
  "no_repeat_ngram_size": 0,
  "num_beams": 1,
  "num_heads": 12,
  "num_layers": 12,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 0,
  "prefix": null,
  "pruned_heads": {},
  "relative_attention_num_buckets": 32,
  "repetition_penalty": 1.0,
  "

In [ ]:
import json

test = json.load(open("data/MultiWOZ/test.json"))

In [ ]:
test[0].keys()

dict_keys(['dialogue', 'scenario'])

In [ ]:
test[0]['scenario']['kb']['items'][0].keys()

dict_keys(['name', 'address', 'area', 'phone', 'postcode', 'pricerange', 'stars', 'type', 'choice', 'ref', 'parking', 'interent'])

In [ ]:
from tqdm import tqdm

In [ ]:
def generate_dataset(data_split,tokenizer,debugging,domain=None):
    data = []
    number_of_dialogues = 0
    for _,v in tqdm(data_split.items(), total=len(data_split)):
        dialogue = get_dialogue(v['conversation'],tokenizer)
        data.append({'id':v["src"].lower(),'edges':[],'adj_mat':[],'nodes':[], "dialogue":dialogue, "domain":domain})
        number_of_dialogues += 1
        if(debugging and number_of_dialogues >10): break
    return data


In [ ]:
def get_dialogue(dial,tokenizer):
    dialogue = []
    history = []
    for i, d in enumerate(dial):
        if(d['spk']=='USR' or d['spk']=='API'):
            history.append(tokenizer.encode(d["text"],add_special_tokens=False))
        else:
            dialogue.append({"history":list(history),
                             "response":tokenizer.encode(d["text"],add_special_tokens=False),
                             "spk":d['spk']})
            history.append(tokenizer.encode(d["text"],add_special_tokens=False))
    return dialogue

In [ ]:
history = []
dialogue = []
for i, d in enumerate(test):
    # print(d['dialogue'][i]['turn'])
    # print(d['dialogue'][len(d)]['turn'])

    if d['dialogue'][len(d)]['turn'] == 'user':
        history.append(d['dialogue'][len(d)]['utterance'])
    elif d['dialogue'][len(d)]['turn'] == 'system':
        dialogue.append({"history": list(history),
        "response": d['dialogue'][len(d)]['utterance']})

        

In [ ]:
history

['i am looking for a cheap place to stay including free parking',
 'how about a place in the east side of town that is a hotel not a guesthouse',
 'i am looking for a guesthouse with free wifi and parking , please .',
 'sorry , actually i need an expensive restaurant in the north . the first on your list would be great .',
 "thanks ! that ' s all i needed .",
 'i would like expensive , please .',
 'do you have a favorite you could recommend ? i will need the phone and postcode and food type also please .',
 '5 people on a wednesday at 19:45 .',
 'i would like a restaurant in the centre .',
 'the rating should be 4 star and i want free parking to be included .',
 'yes please ! for 5 people and for 5 nights . starting wednesday .',
 'north area in a guesthouse',
 "no , i just realized i ' ll be in the south area . are there any that meet my criteria in the south ?",
 'not in any specific area please . i need to book a table for 3 at 14:00 on saturday .',
 "yes , may i have the address , 

In [ ]:
history = []
for i, d in enumerate(test):
    history.append(d['dialogue'][:])
    history.append(d['dialogue'][1::2])

In [ ]:
history

[[{'turn': 'user',
   'utterance': "i wasn ' t planning to stay in town tonight , but it looks like i ' m going to have to . can you help me find a pretty cheap room ?"},
  {'turn': 'user',
   'utterance': 'i am looking for a cheap place to stay including free parking'},
  {'turn': 'user',
   'utterance': "how many stars does it have ? i ' d like a cheap place in the west with 4 star and free parking ."},
  {'turn': 'user', 'utterance': 'do they have internet there ?'},
  {'turn': 'user', 'utterance': 'sure you can book that for me , thank you !'},
  {'turn': 'user',
   'utterance': "actually , i ' m not ready to book yet . i think i have the information i need , thank you !"}],
 [{'turn': 'system',
   'utterance': 'the the cambridge belfry is a hotel in the west part of town in the cheap price range . would you like me to book a room for you ?',
   'annotated_query': 'find a hotel in the cheap price range',
   'annotated_knowledge': [{'name': 'finches bed and breakfast',
     'address

In [ ]:
#02468 user
#13579 system

In [ ]:
history = []
for i, d in enumerate(test):
    for g,h in enumerate(d):
        print(g)
    #print(d['dialogue'][0])

0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1


In [ ]:
history= []
for i, d in enumerate(test):

    if (d['dialogue'][i]['turn']=="user" or d['dialogue'][i]['turn']=="system"):
        history.append()


0
-----
user
-----
1
-----
system
-----
2
-----
user
-----
3
-----
system
-----
4
-----


IndexError: list index out of range

In [ ]:
import string
def preprocess_text(text):
    """Preprocess utterance and table value."""
    text = text.strip().replace("\t", " ").lower()
    for p in string.punctuation:
        text = text.replace(p, f" {p} ")
    text = " ".join(text.split())
    return text


In [ ]:
# dialogue = {}
# context = []

# for dial in test:
#     context = []
#     for turn in dial["dialogue"]:
#         if (turn["turn"] =='user'):
#             utt = preprocess_text(turn["utterance"])
#             context.append(utt)
#             dialogue['user'] = utt
#         elif (turn["turn"] =='system'):
#             dialogue['response'] = turn['utterance']
#         else:
#             pass
#         dialogue["spk"] = turn["turn"]

In [ ]:
def get_dialogue(dial,tokenizer):
    dialogue = {}
    context = []
    for dial in test:
        context = []
        for turn in dial["dialogue"]:
            if (turn["turn"] =='user'):
                utt = preprocess_text(turn["utterance"])
                context.append(utt)
                dialogue['user'] = utt
            elif (turn["turn"] =='system'):
                dialogue['response'] = turn['utterance']
            else:
                pass
            dialogue["spk"] = turn["turn"]

    return dialogue, context

In [ ]:
def get_dialogue_tokenizer(data,tokenizer):
    dialogue = {}
    context = []
    for dial in data:
        for turn in dial["dialogue"]:
            if (turn["turn"] =='user'):
                utt = preprocess_text(turn["utterance"])
                context.append(tokenizer.encode(utt), add_special_tokens=False)
            
            elif (turn["turn"] == 'system'):
                dialogue["response"] = tokenizer.encode(turn['utterance'],add_special_tokens=False),
            else:
                pass
            dialogue["spk"] = turn["turn"]
    return dialogue

In [ ]:
def generate_dataset(data_split,tokenizer,debugging,domain=None):
    data = []
    number_of_dialogues = 0
    for _,v in tqdm(data_split.items(), total=len(data_split)):
        dialogue = get_dialogue(v['conversation'],tokenizer)
        data.append({'id':v["src"].lower(),'edges':[],'adj_mat':[],'nodes':[], "dialogue":dialogue, "domain":domain})
        number_of_dialogues += 1
        if(debugging and number_of_dialogues >10): break
    return data

In [ ]:
def generate_dataset(data_split, tokenizer, debugging, domain=None):
    data = []
    number_of_dialogues = 0

    for _, v in tqdm(data_split, total=len(data_split)):
        dialogue = get_dialogue(data, tokenizer)
        data.append({'id':v, 'dialogue':dialogue, "domain":domain})
        number_of_dialogues += 1
        if (debugging and number_of_dialogues > 10): break

    return data

In [ ]:
def get_loader(args,data_raw, tokenizer):
    # if not os.path.isfile("data/MultiWOZ_2.1/train_segment.p"):
    print("Building Sequences")
    # datasets = {"train": defaultdict(list), "valid": defaultdict(list)}
    datasets = {"train": [], "valid": [], "test": []}
    # domains, particularly for MWOZ
    domains = {"train": [], "valid": [], "test": []} 
    
    for dataset_name, data in data_raw.items():
        for conv in tqdm(data,total=len(data)):
            for sample in conv['dialogue']: 
                instance = build_input_from_segments(args,sample['history'], sample["response"], sample["graph"] if args.dataset == "DIALKG" else conv, tokenizer, lm_labels=True, with_eos=True, valid=dataset_name)   
                # for input_name, input_array in instance.items():
                datasets[dataset_name].append(instance)
                if args.dataset == "MWOZ" or args.dataset == "MWOZ_SINGLE":
                    domains[dataset_name].append(conv['domain'])

    print("Build train and validation dataloaders")
    # train_dataset, valid_dataset = TensorDataset(*tensor_datasets["train"]), TensorDataset(*tensor_datasets["valid"])
    train_dataset = DatasetTrain(datasets["train"], domains["train"])
    valid_dataset = DatasetTrain(datasets["valid"], domains["valid"])
    test_dataset = DatasetTrain(datasets["test"], domains["test"])

    if args.balance_sampler:
        print("setup imbalanced sampler")
        train_loader = DataLoader(train_dataset, sampler=ImbalancedDatasetSampler(train_dataset), batch_size=args.train_batch_size, collate_fn=collate_fn)
    else:
        train_loader = DataLoader(train_dataset, batch_size=args.train_batch_size, shuffle=True,collate_fn=collate_fn)
    
    valid_loader = DataLoader(valid_dataset, batch_size=args.valid_batch_size, shuffle=False,collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=args.valid_batch_size, shuffle=False,collate_fn=collate_fn)

    print("Train: {}".format(len(train_dataset)))
    print("Valid: {}".format(len(valid_loader)))
    print("Test: {}".format(len(test_loader)))
    return train_loader, valid_loader, test_loader


In [ ]:
# def build_input_from_segments(args, history, reply, graph, tokenizer, lm_labels=False, with_eos=True, valid="train"):
#     """ Build a sequence of input from 3 segments: persona, history and last reply. """

#     bos, eos, speaker1, speaker2 = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[:-1])
#     if(args.flatten_KB and with_eos): history += graph['edges']
#     history = history[-args.max_history:]
#     sequence = [[bos]] + history + [reply + ([eos] if with_eos else [])]
#     sequence = [sequence[0]] + [[speaker2 if (len(sequence)-i) % 2 else speaker1] + s for i, s in enumerate(sequence[1:])]
#     instance = {}
#     instance["input_ids"] = list(chain(*sequence))
#     instance["token_type_ids"] = [speaker2 if i % 2 else speaker1 for i, s in enumerate(sequence) for _ in s]
#     instance["lm_labels"] = [-100] * len(instance["input_ids"])

#     instance["input_graph_ids"] = None
#     instance["input_graph_networks"] = None
#     if args.edge_list:
#         instance["input_graph_ids"] = graph['edges']
#     elif(args.graph):
#         instance["input_graph_ids"] = graph['nodes']
#         if(args.adj_graph):
#             instance["input_graph_networks"] = graph['adj_mat']

#     if lm_labels:
#         instance["lm_labels"] = ([-100] * sum(len(s) for s in sequence[:-1])) + [-100] + sequence[-1][1:]

#     ### attetion mask
#     instance["attention_mask"] = None
#     instance["len_token_a"] = 0
#     instance["padding_token"] = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[-1])
#     if(args.unilm):
#         len_seq = len(instance["lm_labels"])    
#         input_mask = torch.zeros(len_seq, len_seq, dtype=torch.long)
#         tril_matrix = torch.tril(torch.ones((len_seq, len_seq), dtype=torch.long))
#         len_token_a = len(list(chain(*history))) + len(history) + 1 # +1 BOS and len(history) account for the special token in between the turns   
#         len_token_b = len(list(chain(*[reply + ([eos] if with_eos else [])]))) + 1
#         assert len_token_a + len_token_b == len(instance["lm_labels"])
#         input_mask[:len_token_a + len_token_b+1, :len_token_a+1].fill_(1)
#         second_st, second_end = len_token_a, len_token_a+len_token_b
#         input_mask[second_st:second_end, second_st:second_end].copy_(tril_matrix[:second_end-second_st, :second_end-second_st])
#         # from numpy import save
#         # save('unilm.npy', input_mask.cpu().numpy())
#         instance["attention_mask"] = input_mask
#         instance["len_token_a"] = len_token_a
#     return instance

In [ ]:
def get_loader(args,data, tokenizer):
    print("Building Sequences")

    # datasets = {"train": defaultdict(list), "valid": defaultdict(list)}
    datasets = {"train": [], "valid": [], "test": []}
    # domains, particularly for MWOZ
    domains = {"train": [], "valid": [], "test": []} 

    for i ,dial in tqdm(enumerate(data, total=len(data))):
        instance = 
    

In [ ]:

def load_MWOZ(args,tokenizer,test_flag=False,debugging=False):
    train = json.load(open("data/MultiWOZ_2.1/train_data.json"))
    valid = json.load(open("data/MultiWOZ_2.1/valid_data.json"))
    test = json.load(open("data/MultiWOZ_2.1/test_data.json"))
    if(test_flag):
        test = generate_dataset(test,tokenizer,debugging=debugging)
        return None, None, test, None
    else:
        train = generate_dataset(train,tokenizer,debugging=debugging)
        dev = generate_dataset(valid,tokenizer,debugging=debugging)
        test = generate_dataset(test,tokenizer,debugging=debugging)
        dataset_dict = {"train":train,"valid":dev,"test":test}
        train_loader, valid_loader, test_loader = get_loader(args, dataset_dict, tokenizer)
        print(f"Max Len:{test_dataloader(args,train_loader)}")
        print(f"Max Len:{test_dataloader(args,valid_loader)}")
        print(f"Max Len:{test_dataloader(args,test_loader)}")
        return train_loader, valid_loader, None


In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class DatasetTrain(Dataset):
    def __init__(self,data, domains=None):
        self.data = data
        self.dataset_len = len(self.data)
        self.domains = domains


    def __getitem__(self, index):
        item = self.data[index]
        return item

    def __len__(self):
        return self.dataset_len

In [3]:
from utils import definition

def init_tokenizer(backbone):
    tokenizer = T5Tokenizer.from_pretained(backbone)
    special_tokens = []

    special_tokens.extend(definition.SPECIAL_TOKENS)

    #add Nothing
    tokenizer.add_special_tokens({"additional_special_tokens": special_tokens})

    return tokenizer

In [7]:
from urllib import request
import json

response = request.urlopen("https://jsonplaceholder.typicode.com/users")
json_response = response.read()
users = json.loads(json_response)

In [8]:
users

[{'id': 1,
  'name': 'Leanne Graham',
  'username': 'Bret',
  'email': 'Sincere@april.biz',
  'address': {'street': 'Kulas Light',
   'suite': 'Apt. 556',
   'city': 'Gwenborough',
   'zipcode': '92998-3874',
   'geo': {'lat': '-37.3159', 'lng': '81.1496'}},
  'phone': '1-770-736-8031 x56442',
  'website': 'hildegard.org',
  'company': {'name': 'Romaguera-Crona',
   'catchPhrase': 'Multi-layered client-server neural-net',
   'bs': 'harness real-time e-markets'}},
 {'id': 2,
  'name': 'Ervin Howell',
  'username': 'Antonette',
  'email': 'Shanna@melissa.tv',
  'address': {'street': 'Victor Plains',
   'suite': 'Suite 879',
   'city': 'Wisokyburgh',
   'zipcode': '90566-7771',
   'geo': {'lat': '-43.9509', 'lng': '-34.4618'}},
  'phone': '010-692-6593 x09125',
  'website': 'anastasia.net',
  'company': {'name': 'Deckow-Crist',
   'catchPhrase': 'Proactive didactic contingency',
   'bs': 'synergize scalable supply-chains'}},
 {'id': 3,
  'name': 'Clementine Bauch',
  'username': 'Samantha

In [9]:
from pprint import pprint
pprint(users)

[{'address': {'city': 'Gwenborough',
              'geo': {'lat': '-37.3159', 'lng': '81.1496'},
              'street': 'Kulas Light',
              'suite': 'Apt. 556',
              'zipcode': '92998-3874'},
  'company': {'bs': 'harness real-time e-markets',
              'catchPhrase': 'Multi-layered client-server neural-net',
              'name': 'Romaguera-Crona'},
  'email': 'Sincere@april.biz',
  'id': 1,
  'name': 'Leanne Graham',
  'phone': '1-770-736-8031 x56442',
  'username': 'Bret',
  'website': 'hildegard.org'},
 {'address': {'city': 'Wisokyburgh',
              'geo': {'lat': '-43.9509', 'lng': '-34.4618'},
              'street': 'Victor Plains',
              'suite': 'Suite 879',
              'zipcode': '90566-7771'},
  'company': {'bs': 'synergize scalable supply-chains',
              'catchPhrase': 'Proactive didactic contingency',
              'name': 'Deckow-Crist'},
  'email': 'Shanna@melissa.tv',
  'id': 2,
  'name': 'Ervin Howell',
  'phone': '010-692-6593